# Структурированный вывод с GigaChat

В данном ноутбуке будет рассмотрено, как получать от GigaChat результат в структурированном формате.

## Установка зависимостей

Для работы примера установите библиотеку langchain-gigachat:

## Инициализация GigaChat

Для работы с моделями GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [ ]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

from langchain_gigachat.chat_models.gigachat import GigaChat

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

model = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
)

## С использованием метода ```.with_structured_output()```

GigaChat поддерживает вызов функций, поэтому самым простым и надёжным способом получения структурированного вывода является метод ```.with_structured_output()```. Для описания схемы вывода можно использовать Pydantic класс (наиболее рекомендованный способ), TypedDict или непостредственно саму json схему функции.

### Pydantic class

Модель может возвращать объект pydantic. Для этого нам необходимо передать нужный класс pydantic. Главное преимущество использования Pydantic заключается в том, что сгенерированный моделью вывод будет проверен. Pydantic выдаст ошибку, если какие-либо обязательные поля отсутствуют или если какие-либо поля имеют неправильный тип. Помимо написания самой структуры pydantic класса, важно дать понятное название классу, добавить его описание (docstring), а также названия и описания всех атрибутов, поскольку вся эта информация пойдёт модели в промпт.

In [3]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic 
class Book(BaseModel):
    """Книга, которая нужна пользователю"""
    title: str = Field(description="Название книги")
    author: Optional[str] = Field(
        default="неизвестен", description="Автор книги"
    )
    language: Optional[str] = Field(
        default="любой", description="Язык написания книги"
    )


structured_llm = model.with_structured_output(Book)

structured_llm.invoke("Найди '1984' Оруэлла в английском оригинале")

Book(title='1984', author='George Orwell', language='English')

### TypedDict

Если явная проверка аргументов вам не нужна, можно определить схему с использованием класса TypedDict. Можно опционально использовать специальный Annotated синтаксис, поддерживаемый LangChain, который позволяет указать значение по умолчанию и описание поля, однако нужно понимать, что значение по умолчанию не заполняется автоматически, если модель его не генерирует, оно используется только при определении схемы, которая передается в модель.

In [4]:
from typing import Optional

from typing_extensions import Annotated, TypedDict


# TypedDict
class Book(TypedDict):
    """Книга, которая нужна пользователю"""
    title: Annotated[str, ..., "Название книги"]

    # Также можно было бы указать title как:

    # title: str                    # no default, no description
    # title: Annotated[str, ...]    # no default, no description
    # title: Annotated[str, "foo"]  # default, no description
    
    author: Annotated[Optional[str], "неизвестен", "Автор книги"]
    language: Annotated[Optional[str], "любой", "Язык написания книги"]

structured_llm = model.with_structured_output(Book)

structured_llm.invoke("Ищу 'Властелин колец' Толкина на английском")

{'author': 'Толкин', 'language': 'английский', 'title': 'Властелин колец'}

### Json Schema

Помимо этого, можно передать напрямую словарь с json схемой. Это не требует импорта или классов и делает более показательным, как именно документируется каждый параметр.

In [5]:
json_schema = {
          "name": "book",
          "description": "Книга, которая нужна пользователю",
          "parameters": {
              "properties": {
                  "title": {
                      "description": "Название книги",
                      "type": "string"
                  },
                  "author": {
                      "description": "Автор книги",
                      "type": "string",
                      "deafault": "неизвестен"
                  },
                  "language": {
                      "description": "Язык написания книги",
                      "type": "string"
                  }
              },
              "required": [
                  "title",
              ],
              "type": "object"
          },
      }
structured_llm = model.with_structured_output(json_schema)

structured_llm.invoke("Нужна 'Краткая история времени' Хокинга на русском")

{'author': 'Стивен Хокинг',
 'language': 'русский',
 'title': 'Краткая история времени'}

## Добавить few-shots

Для более сложных схем очень полезно добавлять в промпт фью-шоты. Это можно сделать несколькими способами:

### Напрямую в системный промпт

Можно добавлять примеры вопросов и ответов напрямую в системный промпт

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system = """Верни из запроса пользователя информацию о книге, которую он хочет найти. Ответ должен содержать title, author и language. 

example_user: Мне нужна книга 'Война и мир'
example_assistant: {{"title": "Война и мир", "author": "неизвестен", "language": "любой"}}

example_user: Нужна сказка 'Маленький принц' Антуана де Сент-Экзюпери на французском.
example_assistant: {{"title": "Маленький принц", "author": "Антуан де Сент-Экзюпери", "language": "французский"}}

example_user: Хочу прочитать 'Сто лет одиночества' Габриэля Гарсиа Маркеса
example_assistant: {{"title": "Сто лет одиночества", "author": "Габриэль Гарсиа Маркес", "language": "любой"}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("Нужна 'Тень ветра' Карлоса Руиса Сафона на испанском")

{'author': 'Карлос Руис Сафон', 'language': 'испанский', 'title': 'Тень ветра'}

### Как вызовы инструмента

Если в основе структурирования вывода используется метод вызова инструментов, мы можем передавать примеры в виде явных вызовов этих инструментов. 

In [7]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Мне нужна книга 'Война и мир'", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "book",
                "args": {
                            "title": "Война и мир",
                            "author": "неизвестен",
                            "language": "любой"
                        },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    HumanMessage("Нужна сказка 'Маленький принц' Антуана де Сент-Экзюпери на французском.", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "title": "Маленький принц",
                    "author": "Антуан де Сент-Экзюпери",
                    "language": "французский"
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Хочу прочитать 'Сто лет одиночества' Габриэля Гарсиа Маркеса", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "title": "Сто лет одиночества",
                    "author": "Габриэль Гарсиа Маркес",
                    "language": "любой"
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = ""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "Нужен 'Евгений Онегин' Пушкина в оригинале на русском", "examples": examples})

{'author': 'Пушкин', 'language': 'русский', 'title': 'Евгений Онегин'}

## Необработанный вывод

LLM не идеальны для генерации структурированного вывода, особенно когда схемы становятся сложными. Вы можете избежать возникновения исключений и самостоятельно обрабатывать необработанный вывод, передав include_raw=True. Это изменит формат вывода, включив в него необработанный вывод сообщения, проанализированное значение (в случае успеха) и любые полученные ошибки:

In [8]:
structured_llm = model.with_structured_output(Book, include_raw=True)

structured_llm.invoke("Нужен 'Фауст' Гёте в переводе на русский")

{'raw': AIMessage(content='', additional_kwargs={'function_call': {'name': 'Book', 'arguments': {'author': 'Иоганн Вольфганг Гёте', 'language': 'русский', 'title': 'Фауст'}}, 'functions_state_id': '7640eb48-de06-4794-b91c-895ab69ea8f4'}, response_metadata={'token_usage': {'prompt_tokens': 63, 'completion_tokens': 43, 'total_tokens': 106, 'precached_prompt_tokens': 48}, 'model_name': 'GigaChat-2-Max:2.0.28.02', 'x_headers': {'x-request-id': 'a455038c-c222-45ce-b849-3fe727181a72', 'x-session-id': '1ce13963-0697-4b7c-9cc8-6d6d8bb99e2a', 'x-client-id': None}, 'finish_reason': 'function_call'}, id='a455038c-c222-45ce-b849-3fe727181a72', tool_calls=[{'name': 'Book', 'args': {'author': 'Иоганн Вольфганг Гёте', 'language': 'русский', 'title': 'Фауст'}, 'id': '99e5d0c9-b24a-40c1-9df0-d56ab838507b', 'type': 'tool_call'}]),
 'parsed': {'author': 'Иоганн Вольфганг Гёте',
  'language': 'русский',
  'title': 'Фауст'},
 'parsing_error': None}

## Использование промптинга и парсинга вывода напрямую

JSON-режима у GigaChat пока нет (на момент марта 2025 года). Если схема слишком сложная для модели, код может упасть с ошибкой. Для таких случаев можно в промпте напрямую просить модель использовать определенный формат вывода, а затем и использовать парсер (самописный или заготовленный из LangChain) для извлечения структурированного ответа из необработанного вывода модели.

### С использованием ```PydanticOutputParser```

В следующем примере используется встроенный PydanticOutputParser для анализа вывода модели чата, предложенной для соответствия заданной схеме Pydantic. Обратите внимание, что мы добавляем format_instructions непосредственно в приглашение из метода в парсере:


In [9]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(
        ..., description="Рост человека, выраженный в метрах."
    )


class People(BaseModel):
    """Информация обо всех людях в тексте."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

Давайте посмотрим, какая информация отправляется в модель:


In [10]:
query = "Анне 23 года, ее рост 6 футов."

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Информация о человеке.", "properties": {"name": {"description": "Имя человека", "title": "Name", "type": "string"}, "height_in_meters": {"description": "Рост человека, выраженный в метрах.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Информация обо всех людях в тексте.", "properties": {"people": {"items": {"$ref": "#/$defs/Person"}, "title": "P

In [11]:
chain = prompt | model | parser

chain.invoke({"query": query})

People(people=[Person(name='Анна', height_in_meters=1.83)])

### Кастомный парсер

Можно также реализовать парсер самостоятельно

In [12]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(
        ..., description="Рост человека, выраженный в метрах."
    )


class People(BaseModel):
    """Информация обо всех людях в тексте."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ответьте на запрос пользователя. Выведите свой ответ в формате JSON, который "
            "соответствует заданной схеме: \`\`\`json\n{schema}\n\`\`\`. "
            "Обязательно заключите ответ в теги \`\`\`json и \`\`\`"
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

<>:31: SyntaxWarning: invalid escape sequence '\`'
<>:32: SyntaxWarning: invalid escape sequence '\`'
<>:41: SyntaxWarning: invalid escape sequence '\`'
<>:31: SyntaxWarning: invalid escape sequence '\`'
<>:32: SyntaxWarning: invalid escape sequence '\`'
<>:41: SyntaxWarning: invalid escape sequence '\`'
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:31: SyntaxWarning: invalid escape sequence '\`'
  "соответствует заданной схеме: \`\`\`json\n{schema}\n\`\`\`. "
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:32: SyntaxWarning: invalid escape sequence '\`'
  "Обязательно заключите ответ в теги \`\`\`json и \`\`\`"
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:41: SyntaxWarning: invalid escape sequence '\`'
  """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:36: PydanticDeprecatedSince20: The `schema` method is 

In [13]:
query = "Анне 23 года, ее рост 6 футов, а Артему 24 и его рост 6.3 фута."

print(prompt.format_prompt(query=query).to_string())

System: Ответьте на запрос пользователя. Выведите свой ответ в формате JSON, который соответствует заданной схеме: \`\`\`json
{'$defs': {'Person': {'description': 'Информация о человеке.', 'properties': {'name': {'description': 'Имя человека', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'Рост человека, выраженный в метрах.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Информация обо всех людях в тексте.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Обязательно заключите ответ в теги \`\`\`json и \`\`\`
Human: Анне 23 года, ее рост 6 футов, а Артему 24 и его рост 6.3 фута.


In [14]:
chain = prompt | model | extract_json

chain.invoke({"query": query})

<unknown>:2: SyntaxWarning: invalid escape sequence '\`'


[{'people': [{'name': 'Анна', 'height_in_meters': 1.83},
   {'name': 'Артем', 'height_in_meters': 1.92}]}]